# HOME ASSIGNMENT #3: SLACK API - TO GSHEET

## 0. Load Modules

In [2]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os
print("done")

done


In [3]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)

In [4]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [5]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

### TODO #1
Hoàn tất đoạn code sau

In [5]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [6]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [7]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


In [8]:
user_df[user_df.name == 'lovelylife.158']

,user_id,name,display_name,real_name,title,phone,is_bot
23,U01UJ9MUADT,lovelylife.158,,Thu Trang,,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

In [6]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'assignment3-gg.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [24]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1pXjtLvF6yLv33htSd-B8UgoLkV99vdAZPJ1Yj3lhoIY' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

Result: https://docs.google.com/spreadsheets/d/1pXjtLvF6yLv33htSd-B8UgoLkV99vdAZPJ1Yj3lhoIY/edit?usp=sharing


## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [8]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [15]:
channel_ls[2] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'id': 'C01CAMNCJJV',
 'name': 'branding-design',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'created': 1601867790,
 'is_archived': False,
 'is_general': False,
 'unlinked': 0,
 'name_normalized': 'branding-design',
 'is_shared': False,
 'parent_conversation': None,
 'creator': 'U01AVDY7JET',
 'is_ext_shared': False,
 'is_org_shared': False,
 'shared_team_ids': ['T01B7SGGMLJ'],
 'pending_shared': [],
 'pending_connected_team_ids': [],
 'is_pending_ext_shared': False,
 'is_member': False,
 'is_private': False,
 'is_mpim': False,
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': '', 'creator': '', 'last_set': 0},
 'previous_names': [],
 'num_members': 9}

### TODO#3 
* Tìm id của channel #atom-assignment2

In [25]:
for c in range(len(channel_ls)):
    if channel_ls[c]['name']=='atom-assignment2':
        print(channel_ls[c]['id'])


C021FSDN7LJ


### Slack API: List messages trong 1 channel

In [26]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C021FSDN7LJ"} ## This is ID of assignment#2 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [27]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [35]:
msg_ls[24]

{'client_msg_id': '501b9821-d10a-4312-a417-29489cba6e67',
 'type': 'message',
 'text': 'Mình gửi assignment 2 nhé\n<https://github.com/cnhhoang850/atom-assignments>',
 'user': 'U01UJ9M3P1T',
 'ts': '1621044117.003500',
 'team': 'T01B7SGGMLJ',
 'attachments': [{'fallback': 'cnhhoang850/atom-assignments',
   'text': 'Data skills &amp; Quantitative methods - DATAcracy ATOM workshop',
   'title': 'cnhhoang850/atom-assignments',
   'id': 1,
   'color': '24292f',
   'fields': [{'title': 'Language',
     'value': 'Jupyter Notebook',
     'short': True},
    {'title': 'Last updated', 'value': '4 minutes ago', 'short': True}],
   'bot_id': 'B01UQU8E2A3',
   'app_unfurl_url': 'https://github.com/cnhhoang850/atom-assignments',
   'is_app_unfurl': True}],
 'blocks': [{'type': 'rich_text',
   'block_id': 'MOT0D',
   'elements': [{'type': 'rich_text_section',
     'elements': [{'type': 'text', 'text': 'Mình gửi assignment 2 nhé\n'},
      {'type': 'link',
       'url': 'https://github.com/cnhhoang85

In [42]:
"github.com" in msg_ls[24]['text']

True

In [12]:
not_learners_id = ['U01BE2PR6LU']

In [49]:
sub = {'SubTime':[], 'user ID submit':[], 'User ID review':[],'latest reply':[],'link github':[]}
for i in range(len(msg_ls)):
    ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
    user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
    if msg_ls[i]['user'] not in not_learners_id:
        if "github.com" in msg_ls[i]['text']:
            text = msg_ls[i]['text']
            github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
            if len(github_link) > 0: github = github_link[0]
            if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
            if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
            if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
            if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
            print(ts, user, reply_users_count, reply_users, latest_reply, github)

            sub['SubTime'].append(ts)
            sub['user ID submit'].append(user)
            sub['User ID review'].append(reply_users)
            sub['latest reply'].append(latest_reply)
            sub['link github'].append(github)

2021-05-22 12:39:34.015500 U01UTGRMGEQ 1 ['U01UTGRMGEQ'] 2021-05-22 21:18:58.019900 https://github.com/nguyenhonghanh2511/
2021-05-21 11:24:09.003100 U01U6JQB695 1 ['U01UTGRMGEQ'] 2021-05-22 21:18:58.019900 https://github.com/Tenderriver/
2021-05-20 18:19:34.017100 U01KDQJ9FCG 2 ['U01UTGT8CVA', 'U01KDQJ9FCG'] 2021-05-21 11:53:35.005600 https://github.com/phanlinh0803/
2021-05-20 07:19:34.008100 U01UMBX2X18 2 ['U01UTGT8CVA', 'U01KDQJ9FCG'] 2021-05-21 11:53:35.005600 https://github.com/thienphu71/
2021-05-19 21:53:45.005800 U01USGKQ771 1 ['U01UTGT8CVA'] 2021-05-21 09:15:19.000700 https://github.com/TrangNgo1709/
2021-05-17 22:36:43.012800 U01UMJ5AQ0K 1 ['U01UTGT8CVA'] 2021-05-21 09:15:19.000700 https://github.com/duongphannamhung/
2021-05-17 07:28:18.005500 U01UTGRMGEQ 1 ['U01UJ9MUADT'] 2021-05-23 10:19:20.022200 https://github.com/nguyenhonghanh2511/
2021-05-16 04:43:27.006100 U01V00KHHHP 3 ['U01UMC0N5U2', 'U01V00KHHHP', 'U01V082PLN5'] 2021-05-22 13:22:25.016600 https://github.com/satur

In [54]:
len(sub['SubTime'])

33

In [53]:
sub = pd.DataFrame(sub) ## Dùng pandas để convert dictionaries thành bảng
sub.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,SubTime,user ID submit,User ID review,latest reply,link github
0,2021-05-22 12:39:34.015500,U01UTGRMGEQ,[U01UTGRMGEQ],2021-05-22 21:18:58.019900,https://github.com/nguyenhonghanh2511/
1,2021-05-21 11:24:09.003100,U01U6JQB695,[U01UTGRMGEQ],2021-05-22 21:18:58.019900,https://github.com/Tenderriver/
2,2021-05-20 18:19:34.017100,U01KDQJ9FCG,"[U01UTGT8CVA, U01KDQJ9FCG]",2021-05-21 11:53:35.005600,https://github.com/phanlinh0803/
3,2021-05-20 07:19:34.008100,U01UMBX2X18,"[U01UTGT8CVA, U01KDQJ9FCG]",2021-05-21 11:53:35.005600,https://github.com/thienphu71/
4,2021-05-19 21:53:45.005800,U01USGKQ771,[U01UTGT8CVA],2021-05-21 09:15:19.000700,https://github.com/TrangNgo1709/


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [56]:
ws2 = sh.add_worksheet("Assignment#2 Submit",1000,256) #create new worksheet
set_with_dataframe(ws2,sub) # APPEND DATA TO SHEET


Result: https://docs.google.com/spreadsheets/d/1pXjtLvF6yLv33htSd-B8UgoLkV99vdAZPJ1Yj3lhoIY/edit?usp=sharing